In [1]:
import os
from glob import glob

import numpy as np
import pandas as pd

In [2]:
small_file = glob('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/*.csv')
small_file

['/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/ratings.csv',
 '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/tags.csv',
 '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/links.csv',
 '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/movies.csv']

In [3]:
small_ratings = pd.read_csv(small_file[0])

In [4]:
small_ratings_2 = small_ratings.groupby('userId').filter(lambda x: len(x) > 10)
small_ratings_3 = small_ratings_2.groupby('movieId').filter(lambda x: len(x) > 10)
small_ratings_3

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100818,610,159093,3.0,1493847704
100829,610,164179,5.0,1493845631
100830,610,166528,4.0,1493879365
100833,610,168250,5.0,1494273047


In [5]:
small_ratings_3['rating'].sum()

285052.5

In [6]:
df = small_ratings_3.sort_values(by='timestamp')
df

,userId,movieId,rating,timestamp
66719,429,595,5.0,828124615
66678,429,218,4.0,828124615
66679,429,222,4.0,828124615
66669,429,165,4.0,828124615
66667,429,161,5.0,828124615
...,...,...,...,...
39172,272,158238,4.0,1537475893
30174,210,177765,5.0,1537632257
30151,210,122916,4.5,1537632280
30150,210,122906,4.5,1537632293


In [7]:
train, val, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

In [8]:
val_sub1, val_sub2 = np.array_split(val, 2)
test_sub1, test_sub2 = np.array_split(test, 2)

In [9]:
train = train.append(val_sub1)
train = train.append(test_sub1)

In [10]:
val = val_sub2
test = test_sub2

In [11]:
# train.to_csv('/scratch/xg693/train_small.txt', index=False, header=False, sep=' ')
# val.to_csv('/scratch/xg693/val_small.txt', index=False, header=False, sep=' ')
# test.to_csv('/scratch/xg693/test_small.txt', index=False, header=False, sep=' ')

In [12]:
train

,userId,movieId,rating,timestamp
45946,305,16,4.5,1460136042
81409,514,76093,4.0,1533872585
6098,42,2100,4.0,996258715
30043,210,260,5.0,1473705468
33431,226,6323,4.0,1139535404
...,...,...,...,...
18335,116,2915,3.0,1337196211
14268,91,1036,4.0,1112713347
79261,491,8529,5.0,1526672907
37528,253,3083,5.0,1286644217


In [13]:
train_rating_sum = train.groupby('movieId')['rating'].sum() 
train_rating = train_rating_sum / train.groupby('movieId').size()

In [14]:
import operator

sorted_train = dict(sorted(train_rating.items(), key=operator.itemgetter(1), reverse=True))

In [15]:
movie_100 = list(sorted_train.keys())[: 100]

In [16]:
val_new = val[val['movieId'].isin(movie_100)]
test_new = test[test['movieId'].isin(movie_100)]

In [17]:
def MAP_rmse(df):
    cnt = 0
    re = 0

    for i in df['userId'].unique():
        table = df[df['userId']==i]
        temp = table.movieId.to_list()

        for j in temp:
            rating = table[table['movieId']==j].rating.to_list()[0]
            re = re + (rating - sorted_train.get(j))   
            cnt = cnt + 1
            
    print("MAP score: ", str(re / cnt), "\nRMSE score: ", str(re ** 2 / cnt))

In [18]:
# Evaluation for validation 
MAP_rmse(val_new)

MAP score:  -0.06183727308937423 
RMSE score:  1.961634200025611


In [19]:
# Evaluation for test
MAP_rmse(test_new)

MAP score:  -0.048644156009237216 
RMSE score:  1.3203696839288632
